In [19]:
import requests
import os, shutil, re
from bs4 import BeautifulSoup

# establishing session
s = requests.Session()

def load_user_data(page, session):
    url = 'http://cyberleninka.ru/article/c/matematika/%d' % (page)
    r = session.get(url)
    encoding = r.encoding if 'charset' in r.headers.get('content-type', '').lower() else None
    soup = BeautifulSoup(r.content, 'lxml', from_encoding=encoding)
    return soup

if not os.path.exists('./cyberleninka_all/'):
    os.makedirs('./cyberleninka_all/')
    
# clear the directory
for the_file in os.listdir('./cyberleninka_all/'):
    file_path = os.path.join('./cyberleninka_all/', the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)
        
# loading files
page = 1
while True:
    soup = load_user_data(page, s)
    article_list = soup.findAll('div', {'class': 'title'})
    if soup:
        for a in article_list:
            english = r'[A-Z|a-z]+$'
            bad_chars = r'(\w)*(\{[^}]+)|(\$)+|(\\)+|(\/)+(\w)*'
            garbage = r'[.jpg]+|[.png]+|[.bmp]+|[.pdf]+'
            if re.search(english, a.text) or re.search(bad_chars, a.text) or re.search(garbage, a.text):
                continue
            with open('./cyberleninka_all/page_%d.html' % (page), 'a') as output_file:
                output_file.write(a.text.encode('utf8') + '\n')
    else:
        break
    page += 1

KeyboardInterrupt: 

In [20]:
with open('./cyberleninka_all.txt', 'w') as outfile:
    for fname in os.listdir('./cyberleninka_all/'):
        file_path = os.path.join('./cyberleninka_all/', fname)
        with open(file_path) as infile:
            outfile.write(infile.read())